In [2]:
import pandas as pd
import numpy as np
import os

import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

In [3]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [6]:
store = pd.read_csv('store.csv', dtype={'Store':'str'}, 
                    parse_dates={'CompDate':['CompetitionOpenSinceYear','CompetitionOpenSinceMonth']})

In [5]:
os.getcwd()

'/Users/ghostcat/Desktop/Git'

In [7]:
store.drop(['Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], axis=1, inplace=True)

In [8]:
store.isna().sum()

CompDate               0
Store                  0
StoreType              0
Assortment             0
CompetitionDistance    3
dtype: int64

In [9]:
train = pd.read_csv('train.csv', infer_datetime_format=True,
                  parse_dates=[2], usecols=[0,1,2,3,4,5,6],
                    dtype={'StateHoliday':'str','Store':'str'})

In [10]:
train.dtypes

Store                object
DayOfWeek             int64
Date         datetime64[ns]
Sales                 int64
Customers             int64
Open                  int64
Promo                 int64
dtype: object

In [11]:
train = pd.merge(train, store, on='Store')

In [12]:
train['Month'] = train.Date.dt.month
train['Year'] = train.Date.dt.year
train['Week'] = train.Date.dt.week
train['Trend'] = train.sort_values(['Store','Date']).groupby('Store').cumcount()+1

In [13]:
train.CompDate[train.CompDate.str.contains('nan')] = np.NaN

/Users/ghostcat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
train['CompDate'] = pd.to_datetime(train.CompDate)

In [15]:
train['CompActive'] = np.where(train.CompDate <= train.Date, 1, 0)

In [16]:
train.CompActive.fillna(0, inplace=True)
train.CompetitionDistance.fillna(-1, inplace=True)

In [17]:
MeanCust = train.groupby('Store')['Customers'].mean().to_frame()
MeanCust.reset_index('Store', inplace=True)
MeanCust.columns = ['Store','MeanCusts']
train = pd.merge(train, MeanCust, on='Store')

In [18]:
train.sort_values(['Store','Date'], inplace=True)

In [19]:
train.drop(['Customers','CompDate'],axis=1,inplace=True)

In [20]:
train.shape

(1017209, 15)

In [21]:
train = train[((train.Open == 1) & (train.Sales!=0))]

In [22]:
train.shape

(844338, 15)

In [23]:
train.head()

,Store,DayOfWeek,Date,Sales,Open,Promo,StoreType,Assortment,CompetitionDistance,Month,Year,Week,Trend,CompActive,MeanCusts
940,1,3,2013-01-02,5530,1,0,c,a,1270.0,1,2013,1,2,1,467.646497
939,1,4,2013-01-03,4327,1,0,c,a,1270.0,1,2013,1,3,1,467.646497
938,1,5,2013-01-04,4486,1,0,c,a,1270.0,1,2013,1,4,1,467.646497
937,1,6,2013-01-05,4997,1,0,c,a,1270.0,1,2013,1,5,1,467.646497
935,1,1,2013-01-07,7176,1,1,c,a,1270.0,1,2013,2,7,1,467.646497


### Test Data

In [24]:
test = pd.read_csv('test.csv', infer_datetime_format=True,
                  parse_dates=[3], usecols=[0, 1, 2, 3, 4, 5],
                  dtype={'Store':'str'})

In [25]:
test.dtypes

Id                    int64
Store                object
DayOfWeek             int64
Date         datetime64[ns]
Open                float64
Promo                 int64
dtype: object

In [26]:
test['Month'] = test.Date.dt.month
test['Year'] = test.Date.dt.year
test['Week'] = test.Date.dt.week
test['Trend'] = test.sort_values(['Store','Date']).groupby('Store').cumcount()+1

In [27]:
test.isna().sum()  # NA values in the Open column -- shift to

Id            0
Store         0
DayOfWeek     0
Date          0
Open         11
Promo         0
Month         0
Year          0
Week          0
Trend         0
dtype: int64

In [28]:
test.fillna(0, inplace=True)

In [29]:
train_cols = train.loc[:,['Store','CompActive','MeanCusts','StoreType','Assortment','CompetitionDistance']]

In [30]:
train_cols = train_cols.groupby('Store').last().reset_index()

There are 1115 stores in the train set but only 865 in the test set

In [31]:
train_cols.Store.unique().size

1115

In [32]:
train_cols.dtypes

Store                   object
CompActive               int64
MeanCusts              float64
StoreType               object
Assortment              object
CompetitionDistance    float64
dtype: object

In [33]:
train_cols.shape

(1115, 6)

In [34]:
test = pd.merge(test, train_cols, on='Store')

In [35]:
test.isna().sum()

Id                     0
Store                  0
DayOfWeek              0
Date                   0
Open                   0
Promo                  0
Month                  0
Year                   0
Week                   0
Trend                  0
CompActive             0
MeanCusts              0
StoreType              0
Assortment             0
CompetitionDistance    0
dtype: int64

In [36]:
test.shape

(41088, 15)

In [37]:
test.sort_values(['Store','Date'], inplace=True)
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,Month,Year,Week,Trend,CompActive,MeanCusts,StoreType,Assortment,CompetitionDistance
47,40233,1,6,2015-08-01,1.0,0,8,2015,31,1,1,467.646497,c,a,1270.0
46,39377,1,7,2015-08-02,0.0,0,8,2015,31,2,1,467.646497,c,a,1270.0
45,38521,1,1,2015-08-03,1.0,1,8,2015,32,3,1,467.646497,c,a,1270.0
44,37665,1,2,2015-08-04,1.0,1,8,2015,32,4,1,467.646497,c,a,1270.0
43,36809,1,3,2015-08-05,1.0,1,8,2015,32,5,1,467.646497,c,a,1270.0


## Organize the data

In [38]:
predictors = ['Store','DayOfWeek', 'Promo', 'Month', 'Year', 
                     'Week', 'Trend', 'CompActive', 'MeanCusts', 'StoreType', 
                     'Assortment','CompetitionDistance']
Id_store = test.loc[:,['Id','Store']]
y_train = train.loc[:,['Sales','Store']]
X_train = train.loc[:,predictors]
X_test = test.loc[:,predictors]

In [39]:
print(y_train.shape)
print(X_train.shape)
print(X_test.shape)

(844338, 2)
(844338, 12)
(41088, 12)


In [40]:
d_cols = ['Month','Year','DayOfWeek','Week','StoreType','Assortment']

In [41]:
X_train = pd.get_dummies(data=X_train, columns=d_cols, drop_first=True)

In [42]:
X_test = pd.get_dummies(data=X_test, columns=d_cols, drop_first=True)

In [43]:
# Get missing columns in the training test
missing_cols = set(X_train.columns) - set(X_test.columns)
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_test = X_test[X_train.columns]

In [44]:
X_train.head()

,Store,Promo,Trend,CompActive,MeanCusts,CompetitionDistance,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Year_2014,Year_2015,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,Week_18,Week_19,Week_20,Week_21,Week_22,Week_23,Week_24,Week_25,Week_26,Week_27,Week_28,Week_29,Week_30,Week_31,Week_32,Week_33,Week_34,Week_35,Week_36,Week_37,Week_38,Week_39,Week_40,Week_41,Week_42,Week_43,Week_44,Week_45,Week_46,Week_47,Week_48,Week_49,Week_50,Week_51,Week_52,StoreType_b,StoreType_c,StoreType_d,Assortment_b,Assortment_c
940,1,0,2,1,467.646497,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
939,1,0,3,1,467.646497,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
938,1,0,4,1,467.646497,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
937,1,0,5,1,467.646497,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
935,1,1,7,1,467.646497,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [45]:
X_test.head()

,Store,Promo,Trend,CompActive,MeanCusts,CompetitionDistance,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Year_2014,Year_2015,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,Week_18,Week_19,Week_20,Week_21,Week_22,Week_23,Week_24,Week_25,Week_26,Week_27,Week_28,Week_29,Week_30,Week_31,Week_32,Week_33,Week_34,Week_35,Week_36,Week_37,Week_38,Week_39,Week_40,Week_41,Week_42,Week_43,Week_44,Week_45,Week_46,Week_47,Week_48,Week_49,Week_50,Week_51,Week_52,StoreType_b,StoreType_c,StoreType_d,Assortment_b,Assortment_c
47,1,0,1,1,467.646497,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
46,1,0,2,1,467.646497,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
45,1,1,3,1,467.646497,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
44,1,1,4,1,467.646497,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
43,1,1,5,1,467.646497,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [46]:
print(y_train.shape)
print(X_train.shape)
print(X_test.shape)

(844338, 2)
(844338, 81)
(41088, 81)


### Fitting a model for each store with grid search

In [47]:
Sales = []
Id = []
retailers = list(test.Store.unique())

In [ ]:
p_grid = {'colsample_bytree':[.3,.5,.8], 
          'learning_rate':[0.1,0.3],
          'max_depth':[5,8]}

xgbr = xgb.XGBRegressor(objective='reg:linear', n_estimators=100)

tscv = TimeSeriesSplit(n_splits=4)

for i in retailers:  # change this to your heart's content
    preds = []
    
    store_train_X = X_train[X_train.Store==i].iloc[:,1:]
    store_test_X = X_test[X_test.Store==i].iloc[:,1:]
    
    Id.extend(Id_store[Id_store.Store==i].loc[:,'Id'])
    
    store_train_y = y_train[y_train.Store==i].loc[:,'Sales']
    
    gs = GridSearchCV(xgbr, param_grid = p_grid, 
                    scoring = 'neg_mean_squared_error', cv = tscv, n_jobs = -1)
     
    gs.fit(store_train_X, store_train_y)
    
    print('Store number:',i, gs.best_params_, round((-1 * gs.best_score_)**0.5,2))
    
    preds = gs.predict(store_test_X)
    
    Sales.extend(list(preds))

### Submitting the predictions

In [254]:
submission = pd.DataFrame({'Id':Id,'Sales':Sales})

In [257]:
submission.sort_values('Id', inplace=True)

In [258]:
submission.head()

,Id,Sales
0,1,4451.695312
48,2,7521.916992
96,3,8725.833008
144,4,6086.293945
192,5,5775.170898


In [259]:
submission.shape

(41088, 2)

In [260]:
# submission.to_csv('HW4_Submission_Group3_6420_XG_WithLoop.csv',index=False)